In [0]:
%pip install nbformat databricks-sdk[openai]==0.38.0 dspy --quiet
dbutils.library.restartPython()

In [0]:
%run ../prompts/sql_prompts_with_DSPy 

In [0]:
%run ../prompts/ml_prompts_with_DSPy

In [0]:
%run ../prompts/optimization_prompts_with_DSPy

In [0]:
%run ../helper/GradingModule

In [0]:
module = Module(databricks.sdk.WorkspaceClient())

In [0]:
# module.set_module_dict(module_dict_sql)
# x = module.get_module_dict()
# print(x)

In [0]:
candidate_dict = {
  "X": {
    "SQL": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/01-SQL-X",
    "Optimization": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-optimization/Optimization-X",
    "ML": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-ml/04-Machine-Learning-X"
  }
}

human_graded_dict = {
  "X": {
    "SQL": [15,15,15,15,15,25],
    "Optimization": [1,0,1,1,1,1,1,0],
    "ML": [5, 30, 25, 5, 0]
  }
}

module_list = {
  "SQL": module_dict_sql,
  "Optimization": module_dict_optimization,
  "ML": module_dict_ml
}

llm_models = {
  'llm-405B': 'databricks/databricks-meta-llama-3-1-405b-instruct',
  'llm-70B': 'databricks/databricks-meta-llama-3-3-70b-instruct'
}

In [0]:
import time 

model_dict = {}

for k,v in llm_models.items():
  start_time = time.time()
  # dspy.context(lm=dspy.LM(llm_models[k]))
  lm = dspy.LM(llm_models[k])
  dspy.settings.configure(lm=lm)
  error_dict = {}
  answers_dict = {}
  for candidate, values in candidate_dict.items():
    for x, context_path in values.items():
      human_answers_list = human_graded_dict[candidate][x]
      module.set_module_dict(module_list[x])
      results = module.get_error_and_answer_dict(context_path, human_answers_list, "users.abhay_jalisatgi.gen_ai_eval", x, candidate)
      error_dict[x] = results['error_dict']
      answers_dict[x] = results['answers_dict']
  end_time = time.time()
  execution_time = end_time - start_time
  print(f" time taken by model {k} for execution is {execution_time} seconds")
  model_dict[k] = {'error_dict': error_dict, 'answers_dict': answers_dict, 'execution_time': execution_time, 'lm':lm}

In [0]:
model_dict['llm-70B']['error_dict']

In [0]:
model_dict['llm-405B']['error_dict']

In [0]:
model_dict['llm-70B']['answers_dict']['SQL'].display()

In [0]:
model_dict['llm-405B']['answers_dict']['SQL'].display()

In [0]:
model_dict['llm-70B']['lm'].history[-1]